## Original code from the class
- Divides the data into a training set (last quarter of 2018) and a test set (first day of 2019) based on date ranges.

In [54]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv("Bitstamp_BTCUSD_2018_minute.csv", header=1)
# df.index = df.date
# df.drop(["unix", "date", "symbol", "Volume USD"], axis=1, inplace=True)
# df = df.iloc[::-1] # Reverse the order of dates

# BTC_vol = df["Volume BTC"].values
# df_diff = df.diff()
# df_diff["Volume BTC"] = np.log(1 + BTC_vol)

# # Select the last quarter as the training dataset
# date_index = pd.to_datetime(df_diff.index)
# mask_train = pd.Series(date_index).between("2018-10-01", "2018-12-31",
#     inclusive="left")
# df_train = df_diff.loc[mask_train.values].copy(deep=True)
# y_close_train = df.close.loc[mask_train.values]
# train_close = df.loc[mask_train.values].close.values
# df_train["Relative_Close"] = train_close / train_close[0]

# # Select the first day as the test dataset
# mask_test = pd.Series(date_index).between("2018-12-31", "2019-01-01",
#     inclusive="left")
# df_test = df_diff.loc[mask_test.values].copy(deep=True)
# test_close = df.loc[mask_test.values].close.values
# df_test["Relative_Close"] = test_close / train_close[0]
# y_close_test = df.close[mask_test.values]

# print(df_train.columns.values)

# def generate_dataset(df, seq_len):
#     X_list, y_list = [], []
#     for i in range(len(df.index) - seq_len):
#         X_list.append(np.array(df.iloc[i:i+seq_len,:]))
#         y_list.append(df.close.values[i+seq_len])
#     return np.array(X_list), np.array(y_list)

In [55]:
# LAG = 5
# # Add LAG number of observations in training dataset to test dataset
# df_test_LAG = pd.concat((df_train.iloc[-LAG:,:], df_test), axis=0)

# X_train, y_train = generate_dataset(df_train, seq_len=LAG)
# X_test, y_test = generate_dataset(df_test_LAG, seq_len=LAG)

# print(np.mean(y_train))
# print(np.mean((y_train - np.mean(y_train))**2))
# print(np.mean((y_test - np.mean(y_train))**2))

# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam

# tf.keras.utils.set_random_seed(4002)

# model = Sequential()
# model.add(LSTM(8, return_sequences=True))
# model.add(LSTM(8, dropout=0.2))

# model.add(Dense(1))
# model.@@compile@@(optimizer=Adam(learning_rate=0.001), loss='mse')
# model.fit(X_train, y_train, batch_size=64, epochs=30, shuffle=True)

# LSTM_pred = np.squeeze(model.predict(X_test))
# print(np.mean((LSTM_pred - y_test)**2))

## Changed to daily basis

In [56]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [57]:
df = pd.read_csv('Bitcoin Historical Data4.csv')

In [58]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,03/18/2024,"67,594.10","68,389.70","68,920.10","66,601.40",78.07K,-1.17%
1,03/17/2024,"68,391.20","65,314.20","68,857.70","64,605.50",66.07K,4.71%
2,03/16/2024,"65,314.20","69,456.50","70,037.00","64,971.00",75.82K,-5.97%
3,03/15/2024,"69,463.70","71,387.10","72,398.10","65,765.60",148.59K,-2.69%
4,03/14/2024,"71,387.50","73,066.70","73,740.90","68,717.20",109.43K,-2.30%


In [59]:
df.columns = ['date', 'close', 'open', 'high', 'low', 'vol', 'change']

### Data preprocessing

In [60]:
def percentage_to_decimal(s):
    return float(s.strip('%')) / 100

In [61]:
def convert_volume(volume):

    volume_str = str(volume)

    if 'K' in volume_str:
        return float(volume_str.replace('K', '')) * 1e3
    elif 'M' in volume_str:
        return float(volume_str.replace('M', '')) * 1e6
    elif 'B' in volume_str:
        return float(volume_str.replace('B', '')) * 1e9
    else:
        return float(volume_str)

In [62]:
df['vol'] = df['vol'].apply(convert_volume)
df['close'] = df['close'].replace(',', '', regex=True).astype(float)
df['open'] = df['open'].replace(',', '', regex=True).astype(float)
df['high'] = df['high'].replace(',', '', regex=True).astype(float)
df['low'] = df['low'].replace(',', '', regex=True).astype(float)
df['change'] = df['change'].apply(percentage_to_decimal)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4993 non-null   object 
 1   close   4993 non-null   float64
 2   open    4993 non-null   float64
 3   high    4993 non-null   float64
 4   low     4993 non-null   float64
 5   vol     4987 non-null   float64
 6   change  4993 non-null   float64
dtypes: float64(6), object(1)
memory usage: 273.2+ KB


In [64]:
df.index = pd.to_datetime(df['date'])

In [65]:
df.head()

,date,close,open,high,low,vol,change
date,,,,,,,
2024-03-18,03/18/2024,67594.1,68389.7,68920.1,66601.4,78070.0,-0.0117
2024-03-17,03/17/2024,68391.2,65314.2,68857.7,64605.5,66070.0,0.0471
2024-03-16,03/16/2024,65314.2,69456.5,70037.0,64971.0,75820.0,-0.0597
2024-03-15,03/15/2024,69463.7,71387.1,72398.1,65765.6,148590.0,-0.0269
2024-03-14,03/14/2024,71387.5,73066.7,73740.9,68717.2,109430.0,-0.0230


In [66]:
df.drop(["date"], axis=1, inplace=True)

In [67]:
# Reverse the dataset order so it starts with the earliest date
df = df.iloc[::-1]

In [68]:
# We will use data above 2017, because there are some null values in the past
df = df[df.index.year > 2017]

In [69]:
# Prepare the volume and price differences, normalize volume
BTC_vol = df["vol"].values
df_diff = df.diff().dropna()
df_diff["vol"] = np.log(1 + BTC_vol[:-1])  # Shifted by 1 to align with diff

In [70]:
# Normalize the price values between 0 and 1
# max_close = np.max(df_diff['close'])
# min_close = np.min(df_diff['close'])
# df_diff['close'] = (df_diff['close'] - min_close)/(max_close - min_close)

In [71]:
# Assuming df_diff has been created by df.diff().dropna() or similar
# First, align the index of df to match df_diff after dropping NaN values
df_aligned = df.loc[df_diff.index]

# Now, proceed with mask creation and selection
mask_train = df_diff.index < "2018-12-01"
df_train = df_diff.loc[mask_train].copy()

# Use df_aligned to ensure the indexes match
train_close = df_aligned.loc[mask_train, "close"].values

# Since indexes are aligned, this operation should now work without error
df_train["Relative_Close"] = train_close / train_close[0]

In [72]:
mask_test = (df_diff.index >= "2018-12-01") & (df_diff.index < "2019-01-01")  # December 2018 for testing
df_test = df_diff.loc[mask_test].copy()
test_close = df_aligned.loc[mask_test, "close"].values
df_test["Relative_Close"] = test_close / train_close[0]

In [73]:
# Generate dataset function
def generate_dataset(df, seq_len):
    X_list, y_list = [], []
    for i in range(len(df) - seq_len):
        X_list.append(df.iloc[i:(i+seq_len), :].values)
        y_list.append(df["close"].iloc[i + seq_len])
    return np.array(X_list), np.array(y_list)

In [74]:
LAG = 3 # Example: Use the past # days to predict the next day

In [75]:
# Prepare training and test datasets
X_train, y_train = generate_dataset(df_train, LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [76]:
X_train

array([[[ 1.30920000e+03, -4.05600000e+02,  1.38460000e+03, ...,
          1.12699745e+01,  1.26700000e-01,  1.00000000e+00],
        [ 4.02500000e+02,  1.30920000e+03,  1.28900000e+02, ...,
          1.18330578e+01, -7.01000000e-02,  1.02728055e+00],
        [ 2.35000000e+01,  4.02400000e+02, -2.63000000e+01, ...,
          1.15762852e+01, -2.58000000e-02,  1.02887333e+00]],

       [[ 4.02500000e+02,  1.30920000e+03,  1.28900000e+02, ...,
          1.18330578e+01, -7.01000000e-02,  1.02728055e+00],
        [ 2.35000000e+01,  4.02400000e+02, -2.63000000e+01, ...,
          1.15762852e+01, -2.58000000e-02,  1.02887333e+00],
        [ 1.77470000e+03,  2.36000000e+01,  1.71820000e+03, ...,
          1.16170242e+01,  1.15400000e-01,  1.14915854e+00]],

       [[ 2.35000000e+01,  4.02400000e+02, -2.63000000e+01, ...,
          1.15762852e+01, -2.58000000e-02,  1.02887333e+00],
        [ 1.77470000e+03,  2.36000000e+01,  1.71820000e+03, ...,
          1.16170242e+01,  1.15400000e-01,  1.149

In [77]:
np.mean(y_train)

-33.75878787878788

In [78]:
print(np.mean((y_train - np.mean(y_train))**2))

152397.78175610647


### Model 1 (from one-minute example)

In [79]:
# Model architecture (unchanged from the minute-by-minute example)
tf.keras.utils.set_random_seed(4002)
model = Sequential([
    LSTM(8, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(8, dropout=0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

c:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [80]:
# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=30, shuffle=True)

Epoch 1/30


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 162172.1719
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162170.6094 
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162164.2812 
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162162.0156 
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162166.6719 
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162156.8750 
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162160.6719 
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162156.5625 
Epoch 9/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162155.9219 
Epoch 10/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162154.7656 
Epoch 11/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162152.3906 
Epoch 12/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162150.6406 
Epoch 13/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162148.0625 
Epoch 14/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162149.9688 
Epoch 15/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1

In [81]:
# Predict and evaluate
LSTM_pred = np.squeeze(model.predict(X_test))
print("Test MSE:", np.mean((LSTM_pred - y_test)**2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Test MSE: 26740.183165408864


### Model 2 SimpleRNN (64 neurons, ReLU)

In [82]:
tf.keras.utils.set_random_seed(4002)
model = Sequential()
model.add(SimpleRNN(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(SimpleRNN(64, dropout=0.5))
model.add(Dense(1))


model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(X_train, y_train, batch_size=64, epochs=30, shuffle=True)

Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 187257.5625  
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 863us/step - loss: 165624.3750
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - loss: 159743.2188
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - loss: 156775.2031
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step - loss: 153325.7969
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - loss: 149266.1719
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - loss: 145003.5938
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - loss: 141004.9531
Epoch 9/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 137468.4688 
Epoch 10/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 134307.0312 
Epoch 11/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - loss: 131320.9062
Epoch 12/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - loss: 128338.4844
Epoch 13/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 125810.8906
Epoch 14/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 123720.2969 
Epoch 15/30
6/6 ━━

In [83]:
RNN_pred = np.squeeze(model.predict(X_test))
mse = tf.keras.losses.MeanSquaredError()
mse_value = mse(y_test, RNN_pred)
mse_value

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=30167.773>

### Model 3 BN, LSTM(64 neurons, sigmoid)

In [84]:
tf.keras.utils.set_random_seed(4002)
model = Sequential()
model.add(BatchNormalization(axis=-1, epsilon=1e-8))
model.add(LSTM(64, activation='sigmoid', input_shape=(X_train.shape[1], X_train.shape[2]), dropout=0.5))
# model.add(SimpleRNN(64))
model.add(Dense(1))


model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(X_train, y_train, batch_size=64, epochs=100, shuffle=True)

Epoch 1/100


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 162195.9219
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 162181.9219 
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162160.9844 
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162155.1406 
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162133.3281 
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - loss: 162134.7812
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162109.7812 
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162102.1719 
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162088.9531 
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162079.9219 
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162080.2500 
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162036.4375 
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162039.7656 
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 162007.3281 
Epoch 15/100
6/6 ━━━━━━━━━

In [85]:
RNN_pred = np.squeeze(model.predict(X_test))
mse = tf.keras.losses.MeanSquaredError()
mse_value = mse(y_test, RNN_pred)
mse_value

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=26648.998>

### Model 4 MLP (128 ReLU, BN, 64 ReLU)

In [86]:
X_train = df_train.drop('close', axis=1)
y_train = df_train['close']

In [87]:
X_train

,open,high,low,vol,change,Relative_Close
date,,,,,,
2018-01-02,-405.6,1384.6,56.5,11.269975,0.1267,1.000000
2018-01-03,1309.2,128.9,1645.5,11.833058,-0.0701,1.027281
2018-01-04,402.4,-26.3,-335.0,11.576285,-0.0258,1.028873
2018-01-05,23.6,1718.2,587.7,11.617024,0.1154,1.149159
2018-01-06,1774.7,125.9,1454.2,11.863308,-0.1041,1.163900
...,...,...,...,...,...,...
2018-11-26,140.7,-19.4,45.6,13.337396,-0.0916,0.260063
2018-11-27,-220.7,-258.4,8.0,13.255216,0.0631,0.262063
2018-11-28,26.1,471.2,178.6,13.033031,0.0955,0.289099


In [88]:
X_train.values

array([[-4.05600000e+02,  1.38460000e+03,  5.65000000e+01,
         1.12699745e+01,  1.26700000e-01,  1.00000000e+00],
       [ 1.30920000e+03,  1.28900000e+02,  1.64550000e+03,
         1.18330578e+01, -7.01000000e-02,  1.02728055e+00],
       [ 4.02400000e+02, -2.63000000e+01, -3.35000000e+02,
         1.15762852e+01, -2.58000000e-02,  1.02887333e+00],
       ...,
       [ 2.61000000e+01,  4.71200000e+02,  1.78600000e+02,
         1.30330315e+01,  9.55000000e-02,  2.89099301e-01],
       [ 3.99100000e+02,  4.83000000e+01,  2.74200000e+02,
         1.31863536e+01, -9.59000000e-02,  2.91207190e-01],
       [ 3.13000000e+01, -9.95000000e+01, -1.86100000e+02,
         1.29347342e+01, -6.71000000e-02,  2.73801858e-01]])

In [89]:
X_test = df_test.drop('close', axis=1)
y_test = df_test['close']

In [90]:
tf.keras.utils.set_random_seed(4002)
model = Sequential()
model.add(tf.keras.Input(shape=(6,))) #input features
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization(axis=-1, epsilon=1e-8))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))


model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(X_train.values, y_train.values, batch_size=64, epochs=100, shuffle=True)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 151158.1250  
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 149927.6250
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 148758.9844
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 147536.9375
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step - loss: 146194.0469
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 144689.7031 
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 142982.5938
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 141039.6562
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - loss: 138865.6406
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - loss: 136436.4688
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 133735.3594
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 130754.8516
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - loss: 127487.0469
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - loss: 123949.2500
Ep

In [91]:
MLP_pred = np.squeeze(model.predict(X_test.values))
mse = tf.keras.losses.MeanSquaredError()
mse_value = mse(y_test.values, MLP_pred)
mse_value

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=6122.6074>

### Model 5 Scaling X, MLP (128 ReLU, BN, 64 ReLU)

In [92]:
X_train = df_train.drop('close', axis=1).values
y_train = df_train['close'].values
X_test = df_test.drop('close', axis=1).values
y_test = df_test['close'].values

In [93]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [94]:
tf.keras.utils.set_random_seed(4002)
model = Sequential()
model.add(tf.keras.Input(shape=(6,))) #input features
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization(axis=-1, epsilon=1e-8))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))


model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(X_train_scaled, y_train, batch_size=64, epochs=100, shuffle=True)

Epoch 1/100


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 151459.3594  
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - loss: 150303.4062
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - loss: 149193.7656
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 148003.0156 
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - loss: 146664.1250
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - loss: 145113.2031
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 143258.6406 
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - loss: 141037.4375
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - loss: 138399.7812
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - loss: 135303.0938
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - loss: 131703.4062
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 127605.4688
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - loss: 123007.9141
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - loss: 117939.3594
Epoch 15/100
6

In [95]:
X_test_scaled = scaler.transform(X_test)
MLP_pred = np.squeeze(model.predict(X_test_scaled))

mse = tf.keras.losses.MeanSquaredError()
mse_value = mse(y_test, MLP_pred)
mse_value

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=3540.094>

### Model 6 Scaling X, MLP (128 ReLU, BN, 64 ReLU, Dropout)

In [104]:
X_train = df_train.drop('close', axis=1).values
y_train = df_train['close'].values
X_test = df_test.drop('close', axis=1).values
y_test = df_test['close'].values
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

tf.keras.utils.set_random_seed(4002)
model = Sequential()
model.add(tf.keras.Input(shape=(6,))) #input features
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization(axis=-1, epsilon=1e-8))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(X_train_scaled, y_train, batch_size=64, epochs=100, shuffle=True)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 151765.0156  
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 150583.4062 
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 149919.0938 
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - loss: 148696.7500
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - loss: 147857.9531
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - loss: 146987.7969
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - loss: 145295.5781
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 143353.3281
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 142115.5938
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 139609.2500 
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 138212.8750 
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 134272.2344
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 131175.7656
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 127155.3047
Epoch

In [105]:
X_test_scaled = scaler.transform(X_test)
MLP_pred = np.squeeze(model.predict(X_test_scaled))

mse = tf.keras.losses.MeanSquaredError()
mse_value = mse(y_test, MLP_pred)
mse_value

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=3294.976>